根据链家最近7天，最近30天开房记录，判断市场火热度，以及某房型的火热度

In [1]:
import requests,time,os,csv,re,logging
from bs4 import BeautifulSoup

获取html

In [2]:
def get_html(url):
    try:
        r=requests.get(url,timeout=16)
        status=r.status_code
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        now_time=time.strftime('%Y%m%d%H%M%S')
        logging.info(now_time +'      ' +'get_html 异常, ' '    url：'+ url)   

尝试获取n次

In [3]:
def get_n_times(page_url,n=5):
    #获取页面失败或服务器错误时，重试n次
    i=1
    sleep_time=0.1
    index_page=get_html(page_url)
    
    while index_page==None or "小差" in index_page:
        if i > n:
            break
        else:
            time.sleep(sleep_time)
            index_page=get_html(page_url)
            sleep_time+=0.1
            i+=1                
                               
    return index_page

获取某 tag 下总页数

In [22]:
def get_total_page(tag_url):
    page_url=tag_url+'d1'
    index_page=get_n_times(page_url)
    
    if index_page==None or "小差" in index_page:
        total_page=101
    else:
        soup=BeautifulSoup(index_page,'html.parser')
        taged_total_num=soup.find('div',attrs={'class':'resultDes clear'}).span.string   #共找到多少套
#         total_page=int(taged_total_num)//30+1     #每页最多显示30套
#         if total_page>100:
#             total_page=100   #总数超过3000套只显示前3000套
        page_data = soup.find('div',attrs={'class':'page-box house-lst-page-box'})['page-data']
        total_page=eval(page_data)['totalPage']
    return total_page

In [26]:
soup.find('div',attrs={'class':'resultDes clear'}).span.string

' 219 '

页面解析并写入csv

In [49]:
def parse_index_page(date,index_page,tag,page_num,total_page):
    path=os.path.join(os.getcwd(),'results\\%s_index_info_all.csv')  %date #L2：两室，P21:200万以下
    fieldnames=['date','tag','taged_total_num','total_page','page_num',
                'key','title','total_price','total_price_unit','per_price','per_price_unit','prop1']
    if  not os.path.exists(path):     #如果文件不存在，创建文件并写入表头
        csv_file=open(path,'w',newline='')
        writer=csv.DictWriter(csv_file,fieldnames=fieldnames)
        writer.writeheader()
        csv_file.close()
        
    csv_file=open(path,'a+',newline='')
    writer=csv.DictWriter(csv_file,fieldnames=fieldnames)
    
    item={}
    
    soup=BeautifulSoup(index_page,'html.parser')
    
    #get_basic_info
    
    taged_total_num=soup.find('div',attrs={'class':'resultDes clear'}).span.string
    
    
    
    
    #update_basic_info
    item.update({'date':date,'tag':tag,'taged_total_num':taged_total_num,'total_page':total_page,'page_num':page_num})
    
    
    index_info=soup.find_all('a',attrs={'class':'text link-hover-green js_triggerGray js_fanglist_title'})
    for a in index_info:
        key=a['key']
        title=a['title']
        item.update({'key':key,'title':title})
        
        price_div=a.parent.next_sibling.next_sibling
        total_price=price_div.find('span',attrs={'class':'total-price strong-num'}).string
        total_price_unit=price_div.find('span',attrs={'class':'unit'}).string
        per_price_info=price_div.find('span',attrs={'class':'info-col price-item minor'}).string
        per_price_info=re.search('(\d+)(\S{3})',per_price_info).groups()
        per_price=per_price_info[0]
        per_price_unit=per_price_info[1]
        
        prop_div=price_div.next_sibling.next_sibling
        props=prop_div.find_all('span',attrs={'class':'c-prop-tag2'})
        
        if len(props)>0:
            if "距离" in props[0].string:
                prop1=props[0].string
            else:
                prop1=""
        else:
            prop1=""
            
#         prop2=props[1].string
#         prop3=props[2].string
        
        item.update({'total_price':total_price,'total_price_unit':total_price_unit,'per_price':per_price,
                    'per_price_unit':per_price_unit,'prop1':prop1})
        
        writer.writerow(item)    #定义writer2csv函数，每次调用函数写入item,结果是重复写入最后一个item，改用writer写入，无问题
        
    csv_file.close()

In [51]:
aa = soup.find_all('a',attrs={'class':'','target':'_blank'})

In [62]:
aa[12]

<a class="img " data-el="ershoufang" data-housecode="107100029106" data-is_focus="1" data-log_index="3" data-sl="" href="https://sh.lianjia.com/ershoufang/107100029106.html" target="_blank"><div class="focus_tag"></div><img class="video_tag" src="https://s1.ljcdn.com/feroot/pc/asset/img/video.png?_v=20180713180618611"><img alt="文怡花园次新大两房、已满五年、置换急卖、价格可谈！" class="lj-lazy" data-original="https://image1.ljcdn.com/shfdfs-image/20170811/56955eaa-ecd7-41cf-ac8b-629488981a42.jpg.296x216.jpg.232x174.jpg" src="https://s1.ljcdn.com/feroot/pc/asset/img/blank.gif?_v=20180713180618611"/></img></a>

In [63]:
aa[13]

<a class="" data-el="ershoufang" data-housecode="107100029106" data-is_focus="1" data-log_index="3" data-sl="" href="https://sh.lianjia.com/ershoufang/107100029106.html" target="_blank">文怡花园次新大两房、已满五年、置换急卖、价格可谈！</a>

In [67]:
len(soup.find_all('a',attrs={'data-is_focus':'1','data-el':'ershoufang'}))

12

In [ ]:
lambda tag:tag.has_attr('data-id') and tag.has_attr('id')

In [68]:
import re

In [72]:
aaa=(soup.find_all('a',attrs={'data-housecode':re.compile('\d+')}))

In [76]:
aaa[0]

<a class="img " data-el="ershoufang" data-housecode="107002192893" data-is_focus="" data-log_index="1" data-sl="" href="https://sh.lianjia.com/ershoufang/107002192893.html" target="_blank"><img class="video_tag" src="https://s1.ljcdn.com/feroot/pc/asset/img/video.png?_v=20180713180618611"><img alt="5号线延伸线肖塘站 满五唯一税费少" class="lj-lazy" data-original="https://image1.ljcdn.com/shfdfs-image/20171224/b44495fe-0ae7-499e-aa01-597d13bdbef2.jpg.296x216.jpg.232x174.jpg" src="https://s1.ljcdn.com/feroot/pc/asset/img/blank.gif?_v=20180713180618611"/></img></a>

In [78]:
for a in aaa:
    print(a['data-housecode'])

107002192893
107002192893
107000859767
107000859767
107100029106
107100029106
107000678367
107000678367
107100429553
107100429553
107100404897
107100404897
107000678019
107000678019
107000864143
107000864143
107002141163
107002141163
107100084698
107100084698
107100384312
107100384312
107100413346
107100413346
107000864965
107000864965
107002215131
107002215131
107100195484
107100195484
107002144015
107002144015
107002289125
107002289125
107000771309
107000771309
107100266713
107100266713
107000789357
107000789357
107002008665
107002008665
107002104621
107002104621
107002172403
107002172403
107100400391
107100400391
107000794959
107000794959
107002148795
107002148795
107000863529
107000863529
107000863731
107000863731
107100346897
107100346897
107100386743
107100386743
107002192893
107002192893
107000859767
107000859767
107100029106
107100029106
107000678367
107000678367
107100429553
107100429553
107100404897
107100404897
107000678019
107000678019
107000864143
107000864143
107002141163

链接：

西渡/两室/三室：https://sh.lianjia.com/ershoufang/xidu/l2l3/

In [35]:
base_url= 'https://sh.lianjia.com/ershoufang/xidu/l2l3/'
date=time.strftime('%Y%m%d')
# log_dir=os.path.join(os.getcwd(),'results/get_index.log')
# logging.basicConfig(filename=log_dir,level=logging.INFO)

In [36]:
total_page=get_total_page(base_url)

In [32]:
urls = ['https://sh.lianjia.com/ershoufang/xidu/pg'+ str(i) +'l2l3/' for i in range(1,total_page+1)]

In [50]:
parse_index_page(date,index_page,'xidu/l2l3',1,total_page)

IndexError: list index out of range

In [44]:
path=os.path.join(os.getcwd(),'results\\%s_index_info_all.csv')  %date

In [45]:
os.path.exists(path)

False

In [46]:
csv_file=open(path,'w',newline='')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\F Virtual\\Codes\\py\\pycharm\\GitHub\\reptiles_gh\\lianjia\\Lianjia Recent\\results\\20180716_index_info_all.csv'